In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import sys
import sqlite3
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
cwd = os.getcwd()
PROJECT_PATH = cwd.split("quince_assignment")[0] + 'quince_assignment'
sys.path.insert(0, PROJECT_PATH)

In [3]:
event_data = pd.read_csv(
    f"{PROJECT_PATH}/src/data/event_202012182112.csv.gz", compression="gzip")

In [4]:
event_data['event_time'] = pd.to_datetime(event_data['event_time'],
                                          format="%Y-%m-%d %H:%M:%S")

## Overall stats

In [5]:
print(f"Event data has {event_data.shape[0]} rows and {event_data.shape[1]} columns")

Event data has 3638376 rows and 19 columns


In [6]:
min_date = event_data['event_time'].min().strftime('%d%b%Y')
max_date = event_data['event_time'].max().strftime('%d%b%Y')
print(f"Event data is from {min_date} to {max_date}")

Event data is from 15Sep2020 to 01Dec2020


## Column wise analysis

In [7]:
unique_items = event_data.nunique().to_dict()

In [17]:
event_type_order = {'checkout_page_view_contact_information': '4a', 'email_subscription_view': '6', 
                    'checkout_page_view_payment_method': '4c', 'checkout_page_view_stock_problems': '4e', 
                    'checkout_page_view_review': '4f', 'checkout_page_view_processing': '4d', 'add_to_cart': '3', 
                    'collection_view': '1', 'checkout_page_view_forward': '4g', 'product_view': '2', 
                    'checkout_page_view_shipping_method': '4b', 'order_complete': '5', np.NaN: '7'
                   }

In [18]:
event_data = event_data.assign(event_id=event_data['event_type']) 
event_data.replace({'event_id': event_type_order}, inplace=True)

In [11]:
for col in event_data.columns:
    print("="*50)
    print(f"Analysis for {col}")
    print(f"{col} has {unique_items[col]} unique items - {np.round(unique_items[col]/event_data.shape[0]*100, 2)}%")
    if unique_items[col] <= 20:
        print("Printing the frequency of categories")
        print(event_data[col].value_counts(normalize=True).cumsum())
    else:
        print("Printing sample values")
        print(list(event_data[col][0:5]))
    print("="*50)

Analysis for event_id
event_id has 3638376 unique items - 100.0%
Printing sample values
['1', '2', '2', '1', '1']
Analysis for event_type
event_type has 12 unique items - 0.0%
Printing the frequency of categories
product_view                              0.375208
collection_view                           0.737596
email_subscription_view                   0.888763
add_to_cart                               0.942671
checkout_page_view_contact_information    0.963153
checkout_page_view_processing             0.972526
order_complete                            0.981645
checkout_page_view_payment_method         0.990464
checkout_page_view_shipping_method        0.998892
checkout_page_view_stock_problems         0.999749
checkout_page_view_forward                0.999947
checkout_page_view_review                 1.000000
Name: event_type, dtype: float64
Analysis for session_id
session_id has 817575 unique items - 22.47%
Printing sample values
['_3yqjlmvar', nan, '_khkq9os37', '_hze69q9et', '_g

## Session level data

In [25]:
aggregations = {'event_time': ['max', 'min'],
                'event_id': [lambda x: '->'.join(x), 'count']}
session_data = event_data.groupby(['session_id', 'lb_user_id', 'device',
                                   'device_category', 'user_gender', 'user_state']).agg(aggregations)

In [33]:
session_data.reset_index(inplace=True)
session_data.columns = ['session_id', 'lb_user_id', 'device', 'device_category', 'user_gender',
                        'user_state', 'session_end_time', 'session_start_time', 'event_sequence', 'no_of_events']

In [82]:
session_data = session_data.assign(session_duration_sec=(
    session_data['session_end_time']-session_data['session_start_time']).dt.total_seconds())
session_data = session_data.assign(made_purchase=np.where(session_data['event_sequence'].str.contains("6"), "Yes", "No"))
session_data = session_data.assign(visit_count=session_data.groupby('lb_user_id')['session_end_time'].rank(method='min'))
session_data = session_data.assign(bought_on_first_visit=np.where((session_data['visit_count']==1)&(session_data['made_purchase']=='Yes'), 'Yes', 'No'))

In [84]:
session_data[session_data['visit_count']==1]['bought_on_first_visit'].value_counts()

Yes    116872
No      30475
Name: bought_on_first_visit, dtype: int64

In [35]:
print(f"Number of sessions happened is {session_data.shape[0]}")

Number of sessions happened is 256827


In [39]:
session_data['device_category'].value_counts(normalize=True).cumsum()

iPhone           0.569333
macOS            0.767789
Android          0.865622
Windows          0.950628
iPad             0.994599
Chrome OS        0.999537
Generic Linux    0.999961
iPod             0.999981
BlackBerry       0.999992
Unknown          1.000000
Name: device_category, dtype: float64

In [40]:
session_data['user_gender'].value_counts(normalize=True).cumsum()

Female    0.912587
Male      1.000000
Name: user_gender, dtype: float64

In [71]:
session_data['no_of_events'].value_counts(normalize=True).cumsum()

1      0.227402
2      0.448391
3      0.558785
4      0.628914
5      0.680392
6      0.720734
7      0.752417
8      0.781339
9      0.807372
10     0.830189
11     0.849661
12     0.865886
13     0.879834
14     0.892028
15     0.902802
16     0.912155
17     0.920300
18     0.927994
19     0.934851
20     0.940894
21     0.946240
22     0.950955
23     0.955196
24     0.959019
25     0.962531
26     0.965747
28     0.968446
27     0.971117
29     0.973605
30     0.975700
         ...   
123    0.999883
120    0.999891
146    0.999895
220    0.999899
125    0.999903
130    0.999907
122    0.999910
247    0.999914
132    0.999918
117    0.999922
116    0.999926
115    0.999930
112    0.999934
223    0.999938
95     0.999942
337    0.999945
148    0.999949
200    0.999953
319    0.999957
126    0.999961
138    0.999965
179    0.999969
177    0.999973
172    0.999977
171    0.999981
168    0.999984
162    0.999988
153    0.999992
143    0.999996
182    1.000000
Name: no_of_events, Leng

In [64]:
session_data[['session_duration_sec']].describe(percentiles=[0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]).transpose()

,count,mean,std,min,1%,5%,10%,25%,50%,75%,90%,95%,99%,max
session_duration_sec,256827.0,3940.43342,95419.480645,0.0,0.0,0.0,0.0,6.0,53.0,337.0,1431.0,3702.0,11207.48,6144060.0


In [69]:
session_data['made_purchase'].value_counts(normalize=True).cumsum()

False    0.507205
True     1.000000
Name: made_purchase, dtype: float64

In [86]:
session_data.head()

,session_id,lb_user_id,device,device_category,user_gender,user_state,session_end_time,session_start_time,event_sequence,no_of_events,session_duration_sec,made_purchase,visit_count,bought_on_first_visit
0,_00022uznd,_oqprf6nav,desktop,Windows,Male,New York,2020-11-23 02:59:50,2020-11-23 02:59:03,1->1->2,3,47.0,No,10.0,No
1,_0004g0yuv,_m8i0dtjj6,mobile,iPhone,Female,Maryland,2020-11-11 12:25:05,2020-11-11 12:24:56,2->6,2,9.0,Yes,1.0,Yes
2,_000dfqvrj,_kn3otbq1a,mobile,iPhone,Female,Massachusetts,2020-10-04 23:32:01,2020-10-04 23:31:39,2->1,2,22.0,No,1.0,No
3,_000frx5lw,_u6ia8ze08,mobile,iPhone,Female,Virginia,2020-09-20 14:35:27,2020-09-20 13:07:20,1->6->1->2->3->2->3->2,8,5287.0,Yes,1.0,Yes
4,_000q43w78,_an7s8iurw,mobile,iPhone,Female,New York,2020-09-29 13:22:18,2020-09-29 13:22:13,2->6,2,5.0,Yes,1.0,Yes


## User wise aggregation

In [125]:
user_level_aggregation = {
    'session_duration_sec': 'mean', 'no_of_events' : 'mean', 'session_duration_sec' : 'mean', 'visit_count': 'count',
    'made_purchase' : lambda x: (x=="Yes").sum(), 'bought_on_first_visit': lambda x: (x=="Yes").sum(),
    'device': lambda x: set(x), 'device_category': lambda x: set(x), 'user_state': lambda x: set(x)
}

user_data = session_data.groupby(['lb_user_id', 'user_gender']).agg(user_level_aggregation)

In [153]:
user_data.reset_index(inplace=True)
user_data = user_data.iloc[:, -10:]
user_data.columns = ['lb_user_id', 'user_gender', 'avg_session_duration', 'avg_no_of_events', 'times_visited', 
                     'times_purchased', 'bought_on_first_visit', 'devices_used', 'device_categories_used', 'states_used_from']

In [155]:
user_data['bought_on_first_visit'].value_counts()

1    116872
0     31945
Name: bought_on_first_visit, dtype: int64

In [115]:
session_data[session_data['lb_user_id']=='_000fag02q']

,session_id,lb_user_id,device,device_category,user_gender,user_state,session_end_time,session_start_time,event_sequence,no_of_events,session_duration_sec,made_purchase,visit_count,bought_on_first_visit
185838,_q2xaqbg6t,_000fag02q,mobile,iPhone,Female,Florida,2020-11-29 16:41:35,2020-11-29 16:41:27,2->6,2,8.0,Yes,1.0,Yes


In [114]:
session_data[session_data['lb_user_id']=='_000yveacz']

,session_id,lb_user_id,device,device_category,user_gender,user_state,session_end_time,session_start_time,event_sequence,no_of_events,session_duration_sec,made_purchase,visit_count,bought_on_first_visit
17793,_2icjq8j8q,_000yveacz,mobile,iPhone,Female,California,2020-11-14 09:50:03,2020-11-14 09:00:55,1->2->6->1->2->1->1->2->1->2->3->1->1->2->1->1...,84,2948.0,Yes,1.0,Yes
147170,_kneef49ms,_000yveacz,mobile,iPhone,Female,California,2020-11-15 07:22:12,2020-11-15 07:22:12,1,1,0.0,No,2.0,No


## Session wise aggregation